In [2]:
import pandas as pd
import requests, json
from string import ascii_lowercase
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.neighbors import KNeighborsRegressor

In [139]:
header = {
    'User-Agent': 'My User Agent 1.0',
    'From': "@TheFirmPenguin#8408"
}
df = requests.get(r"https://prices.runescape.wiki/api/v1/osrs/mapping",headers=header)
df.json()[1]

{'examine': 'Fabulously ancient mage protection enchanted in the 3rd Age.',
 'id': 10344,
 'members': True,
 'lowalch': 20200,
 'limit': 8,
 'value': 50500,
 'highalch': 30300,
 'icon': '3rd age amulet.png',
 'name': '3rd age amulet'}

In [140]:
def item_setup(item:dict):     
    item_info = {'name': item['name'],
        'icon': item['icon'],
        'id':item['id'],
        'description':item['examine'],
        "members": item['members'],
        "gp_value": item['value'],

        }
    for key in ["lowalch", "limit", "highalch"]:
        if key not in item.keys():
            item_info[key] = 0
        else:
            item_info[key] = item[key]
    return item_info
item_list = {}
for itemz in df.json():
    item = item_setup(itemz)
    item_list[(item['name'])] = item

correct_df = pd.DataFrame.from_dict(item_list)
correct_df
tpose_df = correct_df.transpose().reset_index()
tpose_df.drop('index',axis=1,inplace=True) 
tpose_df

,name,icon,id,description,members,gp_value,lowalch,limit,highalch
0,3rd age 2h axe,3rd age 2h axe.png,28226,"A beautifully crafted axe, shaped by ancient s...",False,1,1,0,1
1,3rd age amulet,3rd age amulet.png,10344,Fabulously ancient mage protection enchanted i...,True,50500,20200,8,30300
2,3rd age axe,3rd age axe.png,20011,"A beautifully crafted axe, shaped by ancient s...",True,55000,22000,40,33000
3,3rd age bow,3rd age bow.png,12424,A beautifully crafted bow carved by ancient ar...,True,150000,60000,8,90000
4,3rd age cloak,3rd age cloak.png,12437,A beautiful cloak woven by ancient tailors.,True,85000,34000,8,51000
...,...,...,...,...,...,...,...,...,...
3998,Zogre bones,Zogre bones.png,4812,A pile of Zombie Ogre bones.,True,1,1,3000,1
3999,Zombie head (Treasure Trails),Zombie head (Treasure Trails).png,19912,Get a head in life.,True,5000,2000,4,3000
4000,Zul-andra teleport,Zul-andra teleport.png,12938,Teleports you to Zul-Andra.,True,10,4,10000,6
4001,Zulrah's scales,Zulrah's scales 5.png,12934,Flakes of toxic snakeskin.,True,20,8,30000,12


After finally getting the dataframe to the way I want it, I need to check to see if any nulls slipped through and the data type of each column.

In [141]:
tpose_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4003 entries, 0 to 4002
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         4003 non-null   object
 1   icon         4003 non-null   object
 2   id           4003 non-null   object
 3   description  4003 non-null   object
 4   members      4003 non-null   object
 5   gp_value     4003 non-null   object
 6   lowalch      4003 non-null   object
 7   limit        4003 non-null   object
 8   highalch     4003 non-null   object
dtypes: object(9)
memory usage: 281.6+ KB


Seeing this, I need to convert id, gp_value, lowalch, limit, and highalch into integers.

In [142]:
for col in ["id", "gp_value", "lowalch", "limit", "highalch"]:
    tpose_df[col] = tpose_df[col].astype(int)
tpose_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4003 entries, 0 to 4002
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         4003 non-null   object
 1   icon         4003 non-null   object
 2   id           4003 non-null   int32 
 3   description  4003 non-null   object
 4   members      4003 non-null   object
 5   gp_value     4003 non-null   int32 
 6   lowalch      4003 non-null   int32 
 7   limit        4003 non-null   int32 
 8   highalch     4003 non-null   int32 
dtypes: int32(5), object(4)
memory usage: 203.4+ KB


Now I can plot graphs to see the distribution of each integer column. I will put in a simple function to post a histogram of each graph for starters.

In [143]:
for col in tpose_df.keys():
    if tpose_df[col].dtype !='O':
        display(px.histogram(x=tpose_df[col],labels={'x':col}))


Looking at these basic graphs, it show that with a few far reaching values causes the main proportion to be squished. To get a better view of the distribution of each column except id, I will be filtering between 1 and each graphs first tick mark to start. The reason I start at 1 is so that if there is a large abundance of 0 value items created by the filter on my cleanup step.

In [144]:
gp_mask = tpose_df[(tpose_df["gp_value"] >0) & (tpose_df["gp_value"] < 1000000)]
gp_graph = px.histogram(x=gp_mask["gp_value"],
                        labels={'x':"Gold Piece Value"}, 
                        title="Items Worth Between 1gp & 1,000,000gp",
                        template = "plotly_dark")
gp_graph.update_yaxes(
    title="Count of Items at Value X"
)
gp_graph.update_layout(
    title_font_color="#0066cc",
    font_color="#0066cc"
)
gp_graph.update_traces(marker={"color":"gold"})

A majority of the items are 10,00gp or else. Next, I want to see the High Alchemy column.

In [145]:
ha_mask = tpose_df[(tpose_df["highalch"] >0) & (tpose_df["highalch"] < 2000000)]
ha_graph = px.histogram(x=ha_mask["highalch"],
                        labels={'x':"Gold Piece Value"}, 
                        title="Items That High Alch Between 1gp & 2,000,000gp",
                        template = "plotly_dark")
ha_graph.update_yaxes(
    title="Count of Items at Value X"
)
ha_graph.update_layout(
    title_font_color="#0066cc",
    font_color="#0066cc"
)
ha_graph.update_traces(marker={"color":"gold"})

In [146]:
low_mask = tpose_df[(tpose_df["lowalch"] >0) & (tpose_df["lowalch"] < 2000000)]
low_graph = px.histogram(x=ha_mask["lowalch"],
                        labels={'x':"Gold Piece Value"}, 
                        title="Items That Low Alch Between 1gp & 2,000,000gp",
                        template = "plotly_dark")
low_graph.update_yaxes(
    title="Count of Items at Value X"
)
low_graph.update_layout(
    title_font_color="#0066cc",
    font_color="#0066cc"
)
low_graph.update_traces(marker={"color":"gold"})

Lastly, I want to look at the correlation between the limit and high alchemy value of each item. This should help partially shine a light on why the limits are the way they are.

In [188]:
hili_mask = tpose_df[(tpose_df["highalch"] >0) & (tpose_df["limit"] > 0)]
hili_graph = px.scatter(x=hili_mask["highalch"], y=hili_mask['limit'],
                        labels={'x':"Gold Piece Return on High Alchemy"}, 
                        title="Correlation Between High Alchemy and the Grand Exchange Buy Limit",
                        template = "plotly_dark")
hili_graph.update_yaxes(
    title="Count of Items at Value X"
)
hili_graph.update_layout(
    title_font_color="#0066cc",
    font_color="#0066cc"
)
hili_graph.update_traces(marker={"color":"gold"})
X = hili_mask.highalch.values.reshape(-1, 1)
x_range = np.linspace(X.min(), X.max(), 4000)

hiya_neighb = KNeighborsRegressor(n_neighbors=400, weights='uniform')
hiya_neighb.fit(X, hili_mask.limit)

y_dist = hiya_neighb.predict(x_range.reshape(-1, 1))
hili_graph.add_traces(go.Scatter(x=x_range, y=y_dist, name='Uniform Weight',
                                    marker = {'color' : '#ff1297'}))

hili_graph.show()


After looking at the original graph, I noticed that it the values were being very asymptote. Where the closer you got to y-axis, the higher the limit. Whereas the further away you were from the y-axis, the lower the limit was. I got inspired to see if I could get a model to draw an asymptote line. 

I took a deep dive into the KneighboursRegression and Polynomial library, and trying out both I decided to stick the Kneighbours. In the kneighbours model, I could choose either uniform weight or distance weight. Because I wanted to get that asymptotic line, I chose uniform so that even every point would contribute to the model line.

If you zoom into the graph, you can see where it actually starts to veer off away from the x-axis until it abruptly stops at the y-intercept.

In [6]:
header = {
            'User-Agent': 'Grabbing Bond Price for App',
            'From': "@TheFirmPenguin#8408"
            }
bond = requests.get(r"https://prices.runescape.wiki/api/v1/osrs/latest?id=13190",headers=header).json()
bond_value = 6.99/((bond['data']['13190']['high']+bond['data']['13190']['low'])/2)
bond_value

8.175438596491228e-07

In [15]:
header = {
            'User-Agent': 'Testing for idea to reduce total querying',
            'From': "@TheFirmPenguin#8408"
            }

cheese = requests.get(r"https://prices.runescape.wiki/api/v1/osrs/latest", headers=header).json()
cheese['data']['28226']

KeyError: '28226'